In [16]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV
import matplotlib.pyplot as plt

In [17]:
data = pd.read_csv('D:/ForMa系统的预测模型训练数据/train-data.txt', sep=' - ', header=None, names=['timestamp', 'load', 'is_peak'], engine='python')

# 转换时间戳为datetime类型
data['timestamp'] = pd.to_datetime(data['timestamp'], format='%H:%M:%S')

# 提取时间特征
data['hour'] = data['timestamp'].dt.hour
data['minute'] = data['timestamp'].dt.minute
data['second'] = data['timestamp'].dt.second

# 计算正余弦特征来保留时间的周期性
data['hour_sin'] = np.sin(2 * np.pi * data['hour'] / 24)
data['hour_cos'] = np.cos(2 * np.pi * data['hour'] / 24)

data['minute_sin'] = np.sin(2 * np.pi * data['minute'] / 60)
data['minute_cos'] = np.cos(2 * np.pi * data['minute'] / 60)

data['second_sin'] = np.sin(2 * np.pi * data['second'] / 60)
data['second_cos'] = np.cos(2 * np.pi * data['second'] / 60)

# 数据标准化（仅对负载进行标准化）
scaler = MinMaxScaler(feature_range=(0, 1))
data[['load']] = scaler.fit_transform(data[['load']])

# 使用负载峰值标识（is_peak）特征
data['is_peak'] = data['is_peak'].astype(float)  # 确保 is_peak 是浮动特征（0 或 1）

In [18]:
# 创建时间序列数据
def create_dataset(dataset, time_step=1, predict_steps=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - time_step - predict_steps + 1):
        # 输入：过去 time_step 个时刻的特征数据
        a = dataset.iloc[i:(i + time_step)].values
        dataX.append(a)
        
        # 输出：未来 predict_steps 个时刻的负载值
        b = dataset.iloc[(i + time_step):(i + time_step + predict_steps), 0].values
        dataY.append(b)
    
    return np.array(dataX), np.array(dataY)

In [19]:
# 设置time_step和预测步数predict_steps
time_step = 60  # 输入时间步数
predict_steps = 60  # 预测未来5个时刻的负载

# 划分训练集和测试集（80%训练，20%测试）
train_size = int(len(data) * 0.8)
train, test = data.iloc[:train_size], data.iloc[train_size:]

In [20]:
# 创建训练集和测试集
features = ['load', 'hour_sin', 'hour_cos', 'minute_sin', 'minute_cos', 'second_sin', 'second_cos', 'is_peak']  # 使用正余弦时间特征
trainX, trainY = create_dataset(train[features], time_step, predict_steps)
testX, testY = create_dataset(test[features], time_step, predict_steps)


In [21]:
# 调整输入数据的形状为 [样本数, 时间步, 特征数]
trainX = trainX.reshape(trainX.shape[0], trainX.shape[1], len(features))
testX = testX.reshape(testX.shape[0], testX.shape[1], len(features))

In [22]:
# 创建LSTM模型
def create_model(neurons=50, dropout_rate=0.2, optimizer='adam', predict_steps=60):
    model = Sequential()
    model.add(LSTM(neurons, return_sequences=True, input_shape=(time_step, len(features))))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(neurons, return_sequences=False))
    model.add(Dense(neurons, activation='relu'))
    
    # 输出未来 predict_steps 个时刻的负载
    model.add(Dense(predict_steps))  # predict_steps 是输出的时间步数
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    
    return model

In [23]:
# 使用KerasRegressor包装模型
model = KerasRegressor(model=create_model, verbose=1)

# 定义超参数分布
param_distributions = {
    'model__neurons': [50, 100],
    'model__dropout_rate': [0.2],
    'model__optimizer': ['adam'],
    'batch_size': [32],
    'epochs': [30, 50]
}

In [24]:
# 进行随机搜索
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_distributions, n_iter=4, scoring='neg_mean_squared_error', cv=3)
random_search_result = random_search.fit(trainX, trainY)


Epoch 1/30
28/28 [==============================] - 5s 43ms/step - loss: 0.0996
Epoch 2/30
28/28 [==============================] - 1s 44ms/step - loss: 0.0488
Epoch 3/30
28/28 [==============================] - 1s 43ms/step - loss: 0.0460
Epoch 4/30
28/28 [==============================] - 1s 42ms/step - loss: 0.0459
Epoch 5/30
28/28 [==============================] - 1s 43ms/step - loss: 0.0459
Epoch 6/30
28/28 [==============================] - 1s 43ms/step - loss: 0.0459
Epoch 7/30
28/28 [==============================] - 1s 43ms/step - loss: 0.0460
Epoch 8/30
28/28 [==============================] - 1s 44ms/step - loss: 0.0458
Epoch 9/30
28/28 [==============================] - 1s 44ms/step - loss: 0.0459
Epoch 10/30
28/28 [==============================] - 1s 43ms/step - loss: 0.0459
Epoch 11/30
28/28 [==============================] - 1s 44ms/step - loss: 0.0459
Epoch 12/30
28/28 [==============================] - 1s 44ms/step - loss: 0.0459
Epoch 13/30
28/28 [==================

In [25]:
# 打印最佳参数和得分
print("Best: %f using %s" % (random_search_result.best_score_, random_search_result.best_params_))


Best: -0.040028 using {'model__optimizer': 'adam', 'model__neurons': 50, 'model__dropout_rate': 0.2, 'epochs': 50, 'batch_size': 32}


In [26]:
# 使用最佳参数进行训练
best_params = random_search_result.best_params_
best_model = create_model(neurons=best_params['model__neurons'],
                          dropout_rate=best_params['model__dropout_rate'],
                          optimizer=best_params['model__optimizer'],
                          predict_steps=predict_steps)

early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
best_model.fit(trainX, trainY, epochs=best_params['epochs'], batch_size=best_params['batch_size'],
               validation_split=0.2, callbacks=[early_stop], verbose=1)

Epoch 1/50
34/34 [==============================] - 6s 71ms/step - loss: 0.0619 - val_loss: 0.0491
Epoch 2/50
34/34 [==============================] - 2s 46ms/step - loss: 0.0364 - val_loss: 0.0444
Epoch 3/50
34/34 [==============================] - 2s 46ms/step - loss: 0.0353 - val_loss: 0.0443
Epoch 4/50
34/34 [==============================] - 2s 47ms/step - loss: 0.0354 - val_loss: 0.0444
Epoch 5/50
34/34 [==============================] - 2s 48ms/step - loss: 0.0353 - val_loss: 0.0437
Epoch 6/50
34/34 [==============================] - 2s 46ms/step - loss: 0.0351 - val_loss: 0.0435
Epoch 7/50
34/34 [==============================] - 2s 45ms/step - loss: 0.0352 - val_loss: 0.0435
Epoch 8/50
34/34 [==============================] - 2s 45ms/step - loss: 0.0349 - val_loss: 0.0434
Epoch 9/50
34/34 [==============================] - 2s 45ms/step - loss: 0.0349 - val_loss: 0.0436
Epoch 10/50
34/34 [==============================] - 2s 45ms/step - loss: 0.0350 - val_loss: 0.0435
Epoch 11/

In [27]:
print("First 5 predictions on train set:")
print(train_predict[:5])

print("First 5 predictions on test set:")
print(test_predict[:5])


First 5 predictions on train set:


NameError: name 'train_predict' is not defined

In [28]:
# 保存模型
best_model.save('D:/Forma_model/lstm model(60to60).h5')

D:\anaconda3\envs\lstm_env\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [1]:
from tensorflow.keras.models import load_model

# 加载保存的模型
model = load_model('D:/Forma_model/lstm model(60to60).h5')  # 替换为你的实际路径
print("模型加载成功！")

模型加载成功！


In [ ]:
%run "D:/Forma_model/model_API/API_service_predict.py"

模型已成功加载！
 * Serving Flask app 'API_service_predict'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://202.114.6.152:5000
Press CTRL+C to quit
 * Restarting with stat
